# <center> Evaluating Numerical Expressions 
    
The goal of this projcet is to use the stack data structure to evaluate complex numerical expressions. The result of this project is make function named evaluate() that can evaluate expressions stored in string.

## Importing Class

In [1]:
from linked_list import LinkedList 

class Stack(LinkedList):

    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self) : 
        ret = self.tail.data 
        if self.length == 1 :
            self.tail = None 
        else : 
            self.tail = self.tail.prev
            self.tail.next = None 
        self.length -= 1 
        return ret 

## Infix and Postfix Notation 

When we write an expression, we use **infix notation**, meaning that we put the operators between the two operands. For a computer, it's much easier to evaluate an expression written in postfix notation. We can evaluate an expression in postfix notation using a stack. 

1. If we find a number, we push that number to the top of the stack.
2. If we find an operator, we pop the top two elements of the stack, perform the operation, and then push back the result. 

## Tokenize the expression into a list of tokens

In [2]:
def tokenize(string) :
    return string.split()

## Processing an Operator 

There is one important detail we need to consider. When we find an operator, we pop the two values on the top of the stack. When we apply the operator to those two elements, we need to make sure we operator those two numbers in the correct order. 

In [8]:
def processes_minus(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result) 
    
def processes_plus(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result) 
    
def processes_times(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result) 
    
def processes_divide(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result) 
    
def processes_pow(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result) 

## Evaluating Postfix Expressions 

To make an algorithm to evaluate an expression in postfix notation. We need to do the following : 

1. Tokenize the expresssion using the tokenize() function
2. Initialize an empty stack
3. For each token, do the following :
    1. if the token ist an operator, call the corresponding function to process it. 
    2. Otherwise (the token is a number) we push that number to the top of the stack. Since each token is a string, We'll need to convert it to a float first
4. return the value that is left in the stack 

In [9]:
def evaluate_postfix(expression) : 
    # Step 1
    tokenized_string = tokenize(expression)
    # Step 2
    stack = Stack() 
    # Step 3
    for token in tokenized_string : 
        if token == "-" : 
            processes_minus(stack)
        elif token == "+" : 
            processes_plus(stack)
        elif token == "*" :
            processes_times(stack)
        elif token == "/" : 
            processes_divide(stack)
        elif token == "**" :
            processes_pow(stack)
        else : 
            stack.push(float(token))
    # Step 4
    return stack.pop()

In [10]:
# Testing function

expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions : 
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Operator Precednece in Infix Notation 

Like before, to simplify tokenizing the expression, we'll assume that the infix expression string contains spaces between any two tokens (even the parentheses). To convert an expression from to postfix, we'll implement the Shunting-yard algorithm. In the Shunting-yard algorithm, we'll need to compare the precedence of the operators. 

In [11]:
precedence = {'+' : 1, '-' : 1, '*' : 2, '/' : 2, '**' : 3}

print(precedence['+'] < precedence['**'])

True


## From Infix to Postfix

This function will implement the Shunting-yard algorithm. It starts by tokenizing the psotfix expression, and then it processes the token one by one using a stack. It builds the bostfix expression by keeping track of a list named postfix, which will contain the list of tokens in psotfix order.

- Opening parenthesis, ( 
    1. Push the token into the stack for later use when we find a closing parenthesis
- Closing parenthesis, )
    1. While the top of the stack isn't an opening parenthesis, (, pop the top element, and append it to the psotfix token list.
    2. Pop the opening parentheses out of the stack at the end 
- Operator, +, -, *, /, ** 
    1. While the top of the stack is also an opertor with a precedence greater than or equal to this operator, pop the top element, and append it to the postfix token list.
    2. Push the current operator to the top of the stack.
- Operand (any number) : 
    1. Append the number to the postfix token list.

### Opening parenthesis

In [29]:
def process_opening_parenthesis(stack) : 
    stack.push("(") 

### Closing parenthesis 

In [30]:
def process_closing_parenthesis(stack, postfix) : 
    while stack.peek() != "(" : 
        postfix.append(stack.pop())
    stack.pop()

### Operator, +, -, *, / or \***

In [36]:
def process_operator(stack, postfix, operator) : 
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator] : 
        postfix.append(stack.pop())
    stack.push(operator)

### Operand(any number)

In [37]:
def process_number(postfix, number) : 
    postfix.append(number)

## Implementing the Shunting-yard Algorithm

We now have all the pieces we need to implement the infix_to_postfix() function that converts an expression from infix notation to postfix notation.

1. We start by splitting the expression into tokens using the tokenize() function.
2. We initialize an empty stack.
3. We initialize an empty postfix token list.
4. Iterate over all tokens, and for each, do the following:
    - If the token is "(", we call the process_opening_parenthesis() function.
    - If the token is ")", we call the process_closing_parenthesis() function.
    - If the token is an operator, we call the process_operator() function.
    - Otherwise, the token is a number, and we call the process_number() function.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the str.join() method to convert the postfix token list into a string.

In [38]:
def infix_to_postfix(expression) :
    tokens = tokenize(expression) 
    stack = Stack() 
    postfix = []
    
    for token in tokens : 
        if token == "(" : 
            process_opening_parenthesis(stack)  
        elif token == ")" :
            process_closing_parenthesis(stack, postfix) 
        elif token in precedence : 
            process_operator(stack, postfix, token)
        else : 
            process_number(postfix, token) 
    
    while len(stack) > 0 :
        postfix.append(stack.pop())
    return " ".join(postfix) 

## Evaluating Infix expressions

In [39]:
def evaluate(expression) : 
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression) 

In [40]:
# Testing 

expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions :
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0


## Final Code 

In [41]:
from linked_list import LinkedList 

class Stack(LinkedList):

    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self) : 
        ret = self.tail.data 
        if self.length == 1 :
            self.tail = None 
        else : 
            self.tail = self.tail.prev
            self.tail.next = None 
        self.length -= 1 
        return ret 

def tokenize(string) :
    return string.split()

def processes_minus(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result) 
    
def processes_plus(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result) 
    
def processes_times(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result) 
    
def processes_divide(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result) 
    
def processes_pow(stack) : 
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result) 

def evaluate_postfix(expression) : 
    # Step 1
    tokenized_string = tokenize(expression)
    # Step 2
    stack = Stack() 
    # Step 3
    for token in tokenized_string : 
        if token == "-" : 
            processes_minus(stack)
        elif token == "+" : 
            processes_plus(stack)
        elif token == "*" :
            processes_times(stack)
        elif token == "/" : 
            processes_divide(stack)
        elif token == "**" :
            processes_pow(stack)
        else : 
            stack.push(float(token))
    # Step 4
    return stack.pop()

def process_opening_parenthesis(stack) : 
    stack.push("(") 

def process_closing_parenthesis(stack, postfix) : 
    while stack.peek() != "(" : 
        postfix.append(stack.pop())
    stack.pop()

def process_operator(stack, postfix, operator) : 
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator] : 
        postfix.append(stack.pop())
    stack.push(operator)

def process_number(postfix, number) : 
    postfix.append(number)

def infix_to_postfix(expression) :
    tokens = tokenize(expression) 
    stack = Stack() 
    postfix = []
    
    for token in tokens : 
        if token == "(" : 
            process_opening_parenthesis(stack)  
        elif token == ")" :
            process_closing_parenthesis(stack, postfix) 
        elif token in precedence : 
            process_operator(stack, postfix, token)
        else : 
            process_number(postfix, token) 
    
    while len(stack) > 0 :
        postfix.append(stack.pop())
    return " ".join(postfix) 

def evaluate(expression) : 
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression) 